In [1]:
from operators import *
from queries import *
from datetime import datetime, timedelta

In [2]:
# init operator
hs_transfer_operator = HSTransferOperator(
    project_id = "logistics-data-staging-flat"
    , credentials = "/Users/s.lafaurie/.config/gcloud/application_default_credentials.json"
    , dest_project_id = "dh-logistics-product-ops"
    , dataset_id = "pricing"
    , env="LOCAL"
)

/Users/s.lafaurie/opt/anaconda3/envs/eda_env/lib/python3.10/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


# INCREMENTAL

In [3]:
# create staging data
hs_transfer_operator.run_dag(
            staging_query = STAGING_QUERY
        , merge_query = MERGE_QUERY
        , staging_table_name = "hs_sa_rdf_orders_stg"
        , production_table_name = "hs_sa_rdf_orders"
        , end_date = datetime.today()
        , days_back = 2
)

Initiating staging task...
Job ID 6f028a86-6e44-4a35-900e-3201085cb7ab successfully executed: 100%|██████████|
Downloading: 100%|██████████|
Loading from BigQuery into Polars successful

        Loaded 273642 rows 
        to BigQuery table hs_sa_rdf_orders_stg 
        in dh-logistics-product-ops:pricing
        
Initiating merging task...
Merge has finished


# BACKFILL

In [39]:
end_date = datetime.today()
start_date = datetime.strptime("2022-01-01", "%Y-%m-%d")


def get_chunks_from_period(start_date, end_date, chunk_size=14):
    period_chunks = []
    date_i = start_date
    while date_i <= end_date:
        start_i = date_i
        end_i =  start_i + timedelta(days=chunk_size)
        period_chunks.append((start_i, min(end_i, end_date)))
        date_i = end_i 
    return period_chunks


date_chunks = get_chunks_from_period(start_date, end_date, 30)  
end_date_chunks  = [x[1] for x in date_chunks]

In [40]:
for end_date_i in end_date_chunks:
    print(f"Running pipeline for {end_date_i.strftime('%Y-%m-%d')}")
    # create staging data
    hs_transfer_operator.run_dag(
                staging_query = STAGING_QUERY
            , merge_query = MERGE_QUERY
            , staging_table_name = "hs_sa_rdf_orders_stg"
            , production_table_name = "hs_sa_rdf_orders"
            , end_date = end_date_i
            , days_back = 30
    )
    print(f"Done\n")


Running pipeline for 2022-05-31
Initiating staging task...
Job ID de4e6726-413f-4ecb-8414-dbc78e2aca6d successfully executed: 100%|██████████|
Query is running:   0%|          |
Downloading: 100%|██████████|
Loading from bigQuery into Polars succesful
Loaded 3822344 rows to BigQuery table hs_sa_rdf_orders_stg in dh-logistics-product-ops:pricing

Initiating merging task...
Merge has finished
Done

Running pipeline for 2022-06-30
Initiating staging task...
Job ID 3e0cd895-552e-4a3e-b510-2d8831035fe3 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
Loading from bigQuery into Polars succesful
Loaded 5052404 rows to BigQuery table hs_sa_rdf_orders_stg in dh-logistics-product-ops:pricing

Initiating merging task...
Merge has finished
Done

Running pipeline for 2022-07-30
Initiating staging task...
Job ID 01e2e1bb-ad3f-4098-8779-f3e959811670 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
Loading from bigQuery into Polars succesful
Loaded 4462465 row